In [17]:
import pandas as pd
import glob
import os
import re

In [18]:
pd.set_option('display.max_columns', None)

In [19]:
base_path = r"C:\Users\HP\PyData-UniConnect-2025-WorldCup-Analysis"  # Project root
data_path = os.path.join(base_path, 'data', 'WorldCup_Stats', '*.csv')
output_path = os.path.join(base_path, 'outputs', 'cleaned_cricket_worldcup.csv')

In [16]:
csv_files = glob.glob(data_path)
print(f"Found {len(csv_files)} CSV files.")

Found 13 CSV files.


In [13]:
import os
folder_path = r"C:\Users\HP\PyData-UniConnect-2025-WorldCup-Analysis\data\WorldCup_Stats"
print(os.listdir(folder_path))

['.ipynb_checkpoints', '1975_Match_Stats.csv', '1979_Match_Stats.csv', '1983_Match_Stats.csv', '1987_Match_Stats.csv', '1992_Match_Stats.csv', '1996_Match_Stats.csv', '1999_Match_Stats.csv', '2003_Match_Stats.csv', '2007_Match_Stats.csv', '2011_Match_Stats.csv', '2015_Match_Stats.csv', '2019_Match_Stats.csv', '2023_Match_Stats.csv']


In [14]:
data_path = r"C:\Users\HP\WorldCup_Stats\*.csv"

In [20]:
print(f"Data path: {data_path}")

Data path: C:\Users\HP\PyData-UniConnect-2025-WorldCup-Analysis\data\WorldCup_Stats\*.csv


In [21]:
csv_files = glob.glob(data_path)
print(f"Found {len(csv_files)} CSV files: {csv_files}")

Found 13 CSV files: ['C:\\Users\\HP\\PyData-UniConnect-2025-WorldCup-Analysis\\data\\WorldCup_Stats\\1975_Match_Stats.csv', 'C:\\Users\\HP\\PyData-UniConnect-2025-WorldCup-Analysis\\data\\WorldCup_Stats\\1979_Match_Stats.csv', 'C:\\Users\\HP\\PyData-UniConnect-2025-WorldCup-Analysis\\data\\WorldCup_Stats\\1983_Match_Stats.csv', 'C:\\Users\\HP\\PyData-UniConnect-2025-WorldCup-Analysis\\data\\WorldCup_Stats\\1987_Match_Stats.csv', 'C:\\Users\\HP\\PyData-UniConnect-2025-WorldCup-Analysis\\data\\WorldCup_Stats\\1992_Match_Stats.csv', 'C:\\Users\\HP\\PyData-UniConnect-2025-WorldCup-Analysis\\data\\WorldCup_Stats\\1996_Match_Stats.csv', 'C:\\Users\\HP\\PyData-UniConnect-2025-WorldCup-Analysis\\data\\WorldCup_Stats\\1999_Match_Stats.csv', 'C:\\Users\\HP\\PyData-UniConnect-2025-WorldCup-Analysis\\data\\WorldCup_Stats\\2003_Match_Stats.csv', 'C:\\Users\\HP\\PyData-UniConnect-2025-WorldCup-Analysis\\data\\WorldCup_Stats\\2007_Match_Stats.csv', 'C:\\Users\\HP\\PyData-UniConnect-2025-WorldCup-Anal

In [22]:
dfs = [pd.read_csv(file) for file in csv_files]

In [23]:
crick_df = pd.concat(dfs, ignore_index=True)

In [24]:
print("\nShape of combined DataFrame:", crick_df.shape)
print("\nFirst few rows of the DataFrame:")
print(crick_df.head())


Shape of combined DataFrame: (528, 18)

First few rows of the DataFrame:
   Unnamed: 0.1  Unnamed: 0        date        venue match_category team_1  \
0             0          11         NaN   Nottingham   League-Match    PAK   
1             1           5         NaN        Leeds   League-Match    EAf   
2             2          12  1975-06-18        Leeds     Semi-Final    ENG   
3             3           8  1975-06-14   Birmingham   League-Match    ENG   
4             4          13         NaN     The Oval     Semi-Final     NZ   

  team_2  team_1_runs  team_1_wickets  team_2_runs  team_2_wickets  \
0     SL        330.0             6.0        138.0             0.0   
1    IND        120.0             0.0        123.0             0.0   
2    AUS         93.0             0.0         94.0             6.0   
3    EAf        290.0             5.0         94.0             0.0   
4     WI        158.0             0.0        159.0             5.0   

                                    

Initial Data Exploration and Cleaning

In [25]:
print("\nDataFrame Info:")
print(crick_df.info())



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0.1     528 non-null    int64  
 1   Unnamed: 0       528 non-null    int64  
 2   date             364 non-null    object 
 3   venue            528 non-null    object 
 4   match_category   528 non-null    object 
 5   team_1           528 non-null    object 
 6   team_2           528 non-null    object 
 7   team_1_runs      518 non-null    float64
 8   team_1_wickets   518 non-null    float64
 9   team_2_runs      513 non-null    float64
 10  team_2_wickets   513 non-null    float64
 11  result           528 non-null    object 
 12  pom              510 non-null    object 
 13  best_batters     250 non-null    object 
 14  best_bowlers     250 non-null    object 
 15  commentary_line  83 non-null     object 
 16  world_cup_year   528 non-null    int64  
 17 

In [26]:
duplicates = crick_df.duplicated().sum()
print(f"\nNumber of duplicate records: {duplicates}")
if duplicates > 0:
    crick_df = crick_df.drop_duplicates()
    print("Duplicates removed. New shape:", crick_df.shape)


Number of duplicate records: 0


In [27]:
print("\nNull values in each column:")
print(crick_df.isnull().sum())


Null values in each column:
Unnamed: 0.1         0
Unnamed: 0           0
date               164
venue                0
match_category       0
team_1               0
team_2               0
team_1_runs         10
team_1_wickets      10
team_2_runs         15
team_2_wickets      15
result               0
pom                 18
best_batters       278
best_bowlers       278
commentary_line    445
world_cup_year       0
host_country         0
dtype: int64


In [28]:
critical_columns = ['date', 'team_1', 'team_2', 'result']
crick_df = crick_df.dropna(subset=critical_columns)
print("Rows with critical nulls removed. New shape:", crick_df.shape)

Rows with critical nulls removed. New shape: (364, 18)


Handle Outliers and Missing Values

In [29]:
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
    return outliers

In [30]:
outliers_runs1 = detect_outliers(crick_df, 'team_1_runs')
outliers_runs2 = detect_outliers(crick_df, 'team_2_runs')
print(f"\nOutliers in team_1_runs:\n{outliers_runs1}")
print(f"\nOutliers in team_2_runs:\n{outliers_runs2}")


Outliers in team_1_runs:
98      1.0
144    45.0
474    29.0
Name: team_1_runs, dtype: float64

Outliers in team_2_runs:
230     45.0
237     32.0
505    383.0
Name: team_2_runs, dtype: float64


 Reason for Removing Invalid Runs:
 Rows with negative team_1_runs or team_2_runs are removed because negative run totals are impossible in cricket, indicating data entry errors. These values would distort analyses (e.g., average runs, match outcomes). Removal is conditional to preserve valid data.

In [31]:
invalid_runs = crick_df[(crick_df['team_1_runs'] < 0) | (crick_df['team_2_runs'] < 0)]
if not invalid_runs.empty:
    print("Invalid runs detected:\n", invalid_runs)
    crick_df = crick_df[(crick_df['team_1_runs'] >= 0) & (crick_df['team_2_runs'] >= 0)]
    print("Invalid runs removed. New shape:", crick_df.shape)

 Reason for Missing Value Imputation:
 -pom: Imputed with 'Unknown' to handle missing player of the match data (common in early World Cups), preserving rows and enabling categorical analysis without bias.
 -team_1_wickets and team_2_wickets: Imputed with 10 (maximum wickets) as missing values likely indicate a team was all out, ensuring numerical columns are usable for calculations.
-best_batters and best_bowlers: Imputed with '[]' to indicate missing stats (common in older matches), maintaining format for parsing and preventing data loss.



In [32]:
crick_df['pom'] = crick_df['pom'].fillna('Unknown')
crick_df['team_1_wickets'] = crick_df['team_1_wickets'].fillna(10)  # Assume all out
crick_df['team_2_wickets'] = crick_df['team_2_wickets'].fillna(10)  # Assume all out
crick_df['best_batters'] = crick_df['best_batters'].fillna('[]')
crick_df['best_bowlers'] = crick_df['best_bowlers'].fillna('[]')

In [33]:
print("\nNull values after imputation:")
print(crick_df.isnull().sum())


Null values after imputation:
Unnamed: 0.1         0
Unnamed: 0           0
date                 0
venue                0
match_category       0
team_1               0
team_2               0
team_1_runs          7
team_1_wickets       0
team_2_runs         12
team_2_wickets       0
result               0
pom                  0
best_batters         0
best_bowlers         0
commentary_line    290
world_cup_year       0
host_country         0
dtype: int64


## Step 3: Handle Outliers and Missing Values

### Reasons for Actions

1. **Outlier Detection for team_1_runs and team_2_runs**:
   - Outlier detection using the IQR method identifies potential data entry errors (e.g., negative or unrealistically high runs). Cricket run totals vary widely (50 to 400+), so statistical outliers are often valid, reflecting high-scoring games or batting collapses. Outliers are printed for inspection but not removed unless invalid to preserve meaningful data.

2. **Removal of Invalid Runs**:
   - Rows with negative team_1_runs or team_2_runs are removed, as negative runs are impossible in cricket and indicate errors. These values would distort analyses (e.g., average runs, match outcomes). Removal is conditional to minimize data loss.

3. **Missing Value Imputation**:
   - **pom**: Imputed with 'Unknown' to handle missing player of the match data (common in early World Cups), preserving rows and enabling categorical analysis without bias.
   - **team_1_wickets and team_2_wickets**: Imputed with 10 (maximum wickets) as missing values likely indicate a team was all out, ensuring numerical columns are usable for calculations.
   - **best_batters and best_bowlers**: Imputed with '[]' to indicate missing stats (common in older matches), maintaining format for parsing and preventing data loss.

Adding New Columns

In [34]:
crick_df['match_status'] = crick_df['result'].apply(lambda x: 'abandoned' if isinstance(x, str) and 'abandoned' in x.lower() else 'played')

In [37]:
def extract_winner(result, team1, team2):
    if isinstance(result, str):
        if 'abandoned' in result.lower():
            return ''
        elif f"{team1} won" in result:
            return team1
        elif f"{team2} won" in result:
            return team2
        else:
            return 'Unknown'
    return ''

crick_df['winning_team'] = crick_df.apply(lambda row: extract_winner(row['result'], row['team_1'], row['team_2']), axis=1)




In [38]:
# 4.3: Split best_batters and best_bowlers
crick_df['best_batter_1'] = ''
crick_df['best_batter_1_runs'] = 0
crick_df['best_batter_2'] = ''
crick_df['best_batter_2_runs'] = 0
crick_df['best_bowler_1'] = ''
crick_df['best_bowler_1_wick'] = 0
crick_df['best_bowler_2'] = ''
crick_df['best_bowler_2_wick'] = 0

def parse_batters(batters_str):
    try:
        batters = eval(batters_str) if isinstance(batters_str, str) and batters_str != '[]' else []
        batter_1, runs_1, batter_2, runs_2 = '', 0, '', 0
        if len(batters) > 0:
            match = re.match(r'(.+) - (\d+) runs', batters[0])
            if match:
                batter_1, runs_1 = match.group(1).strip(), int(match.group(2))
        if len(batters) > 1:
            match = re.match(r'(.+) - (\d+) runs', batters[1])
            if match:
                batter_2, runs_2 = match.group(1).strip(), int(match.group(2))
        return batter_1, runs_1, batter_2, runs_2
    except:
        return '', 0, '', 0

def parse_bowlers(bowlers_str):
    try:
        bowlers = eval(bowlers_str) if isinstance(bowlers_str, str) and bowlers_str != '[]' else []
        bowler_1, wick_1, bowler_2, wick_2 = '', 0, '', 0
        if len(batters) > 0:
            match = re.match(r'(.+) - (\d+)', bowlers[0])
            if match:
                bowler_1, wick_1 = match.group(1).strip(), int(match.group(2))
        if len(batters) > 1:
            match = re.match(r'(.+) - (\d+)', bowlers[1])
            if match:
                bowler_2, wick_2 = match.group(1).strip(), int(match.group(2))
        return bowler_1, wick_1, bowler_2, wick_2
    except:
        return '', 0, '', 0

crick_df[['best_batter_1', 'best_batter_1_runs', 'best_batter_2', 'best_batter_2_runs']] = crick_df['best_batters'].apply(parse_batters).apply(pd.Series)
crick_df[['best_bowler_1', 'best_bowler_1_wick', 'best_bowler_2', 'best_bowler_2_wick']] = crick_df['best_bowlers'].apply(parse_bowlers).apply(pd.Series)

In [40]:
# Step 5: Column Removal
# ----------------------
# Debug: Print columns before dropping
print("\nColumns before dropping:", list(crick_df.columns))

# Conditionally drop commentary_line if it exists
if 'commentary_line' in crick_df.columns:
    crick_df = crick_df.drop(columns=['commentary_line'])
    print("Dropped 'commentary_line' column.")
else:
    print("Column 'commentary_line' not found in DataFrame. Skipping drop.")

# Final verification
print("\nFinal DataFrame Info:")
print(crick_df.info())
print("\nFinal DataFrame Head:")
print(crick_df.head())

# Save the cleaned DataFrame
crick_df.to_csv(output_path, index=False)
print(f"Cleaned DataFrame saved to: {output_path}")


Columns before dropping: ['Unnamed: 0.1', 'Unnamed: 0', 'date', 'venue', 'match_category', 'team_1', 'team_2', 'team_1_runs', 'team_1_wickets', 'team_2_runs', 'team_2_wickets', 'result', 'pom', 'best_batters', 'best_bowlers', 'world_cup_year', 'host_country', 'match_status', 'winning_team', 'best_batter_1', 'best_batter_1_runs', 'best_batter_2', 'best_batter_2_runs', 'best_bowler_1', 'best_bowler_1_wick', 'best_bowler_2', 'best_bowler_2_wick']
Column 'commentary_line' not found in DataFrame. Skipping drop.

Final DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 364 entries, 2 to 527
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0.1        364 non-null    int64  
 1   Unnamed: 0          364 non-null    int64  
 2   date                364 non-null    object 
 3   venue               364 non-null    object 
 4   match_category      364 non-null    object 
 5   team_1          